# Factorización LU secuencial

## Algoritmo de Doolittle

In [49]:
using LinearAlgebra
function LU_Doolittle(A::Matrix{Float64})
    m,n = size(A)
    L = Matrix{Float64}(I, n, n)
    U = copy(A)
    for j = 1:n-1
        for i = j+1:n
            L[i,j] = U[i,j]/U[j,j]
            U[i,j:end] -= L[i,j]*U[j,j:end]
        end
    end
    return L,U
end

LU_Doolittle (generic function with 1 method)

### Ejemplo matriz 3x3 simple

In [46]:
A = [1.0 2.0 3.0; 4.0 5.0 6.0; 7.0 8.0 10.0]
L, U = LU_Doolittle(A)
display(L)
display(U)
display(L*U)

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 4.0  1.0  0.0
 7.0  2.0  1.0

3×3 Matrix{Float64}:
 1.0   2.0   3.0
 0.0  -3.0  -6.0
 0.0   0.0   1.0

3×3 Matrix{Float64}:
 1.0  2.0   3.0
 4.0  5.0   6.0
 7.0  8.0  10.0

## Algoritmo de Crout

In [32]:
using LinearAlgebra
function LU_Crout(A::Matrix{Float64})
    m,n = size(A)
    U = Matrix{Float64}(I, n, n)
    L = copy(A)
    for i = 1:n
        for j = i+1:n
            U[i,j] = L[i,j]/L[i,i]
            L[i:end,j] = L[i:end,j] - U[i,j]*L[i:end,i]
        end
    end
    return L,U
end

LU_Crout (generic function with 1 method)

### Ejemplo

In [33]:
A = [4.0 12.0 -16.0; 12.0 37.0 -43.0; -16.0 -43.0 98.0]
L, U = LU_Crout(A)
display(L)
display(U)
display(L*U)

3×3 Matrix{Float64}:
   4.0  0.0  0.0
  12.0  1.0  0.0
 -16.0  5.0  9.0

3×3 Matrix{Float64}:
 1.0  3.0  -4.0
 0.0  1.0   5.0
 0.0  0.0   1.0

3×3 Matrix{Float64}:
   4.0   12.0  -16.0
  12.0   37.0  -43.0
 -16.0  -43.0   98.0

## PALU

In [91]:
using LinearAlgebra

function PALU(A)
    n = size(A,1)
    P = Matrix{Float64}(I, n, n)
    L = Matrix{Float64}(I, n, n)
    U = copy(A)

    for k = 1:n-1
        # Encuentra el índice del valor máximo en la columna k
        maxval, maxind = findmax(abs.(U[k:end,k]))
        maxind += k - 1

        # Intercambia las filas k y maxind en U y P
        U[[k, maxind],:] = U[[maxind, k],:]
        P[[k, maxind],:] = P[[maxind, k],:]

        for j = k+1:n
            L[j,k] = U[j,k]/U[k,k]
            U[j,k:n] -= L[j,k]*U[k,k:n]
        end
    end

    return P, L, U
end

PALU (generic function with 1 method)

### Ejemplo

In [92]:
A = [4.0 12.0 -16.0; 12.0 37.0 -43.0; -16.0 -43.0 98.0]
P, L, U = PALU(A)
display(L)
display(U)

display(P*A)
display(L*U)

3×3 Matrix{Float64}:
  1.0   0.0       0.0
 -0.75  1.0       0.0
 -0.25  0.263158  1.0

3×3 Matrix{Float64}:
 -16.0  -43.0   98.0
   0.0    4.75  30.5
   0.0    0.0    0.473684

3×3 Matrix{Float64}:
 -16.0  -43.0   98.0
  12.0   37.0  -43.0
   4.0   12.0  -16.0

3×3 Matrix{Float64}:
 -16.0  -43.0   98.0
  12.0   37.0  -43.0
   4.0   12.0  -16.0

# Factorización LU paralelo

## Doolittle


In [34]:
using LinearAlgebra
using Base.Threads

function parallel_LU_Doolittle(A::Matrix{Float64})
    m,n = size(A)
    L = Matrix{Float64}(I, n, n)
    U = copy(A)
    num_threads = Threads.nthreads()
    threads = []
    for j = 1:n-1
        for i = j+1:n
            if length(threads) < num_threads
                push!(threads, Threads.@spawn begin
                    L[i,j] = U[i,j]/U[j,j]
                    U[i,j:end] -= L[i,j]*U[j,j:end]
                end)
            else
                wait(threads[1])
                popfirst!(threads)
            end
        end
    end
    for thread in threads
        wait(thread)
    end
    return L,U
end

parallel_LU_Doolittle (generic function with 1 method)

### Ejemplo matriz 3x3 simple

In [41]:
A = [1.0 2.0 3.0; 4.0 5.0 6.0; 7.0 8.0 10.0]
L, U = parallel_LU_Doolittle(A)
display(L)
display(U)
display(L*U)

3×3 Matrix{Float64}:
 1.0   0.0      0.0
 4.0   1.0      0.0
 7.0  -2.66667  1.0

3×3 Matrix{Float64}:
 1.0    2.0    3.0
 0.0   -3.0   -6.0
 0.0  -14.0  -27.0

3×3 Matrix{Float64}:
 1.0  2.0   3.0
 4.0  5.0   6.0
 7.0  8.0  10.0

## Crout

In [42]:
using LinearAlgebra
using Base.Threads

function parallel_LU_Crout(A::Matrix{Float64})
    m,n = size(A)
    U = Matrix{Float64}(I, n, n)
    L = copy(A)
    num_threads = Threads.nthreads()
    threads = []
    for i = 1:n-1
        for j = i+1:n
            if length(threads) < num_threads
                push!(threads, Threads.@spawn begin
                    U[i,j] = L[i,j]/L[i,i]
                    L[i:end,j] = L[i:end,j] - U[i,j]*L[i:end,i]
                end)
            else
                wait(threads[1])
                popfirst!(threads)
            end
        end
    end
    for thread in threads
        wait(thread)
    end
    return L,U
end

parallel_LU_Crout (generic function with 1 method)

### Ejemplo sencillo 3x3

In [37]:
A = [1.0 2.0 3.0; 4.0 5.0 6.0; 7.0 8.0 10.0]
L, U = parallel_LU_Crout(A)
display(L)
display(U)
display(L*U)

3×3 Matrix{Float64}:
 1.0   0.0   0.0
 4.0  -3.0   0.0
 7.0  -6.0  -1.4

3×3 Matrix{Float64}:
 1.0  2.0   3.0
 0.0  1.0  -1.2
 0.0  0.0   1.0

3×3 Matrix{Float64}:
 1.0  2.0   3.0
 4.0  5.0  15.6
 7.0  8.0  26.8

# Factorización LU por bloques 

## Doolittle

In [52]:
using LinearAlgebra

function is_singular(A)
    return det(A) == 0
end

function block_LU_Doolittle(A)
    n = size(A, 1)
    if n == 1
        return A, A, A
    end

    m = n ÷ 2
    A11 = A[1:m, 1:m]
    A12 = A[1:m, m+1:end]
    A21 = A[m+1:end, 1:m]
    A22 = A[m+1:end, m+1:end]

    if is_singular(A11) || is_singular(A22)
        error("La matriz es singular")
    end

    L11, U11 = LU_Doolittle(A11)
    U12 = L11 \ A12
    L21 = A21 / U11
    L22, U22 = LU_Doolittle(A22 - L21*U12)

    L = [L11 zeros(m, n-m); L21 L22]
    U = [U11 U12; zeros(n-m, m) U22]

    return L, U
end

block_LU_Doolittle (generic function with 1 method)

### Pruebas

In [54]:
A = rand(10,10)
L, U = block_LU_Doolittle(A)

display(L)
display(U)

10×10 Matrix{Float64}:
 1.0         0.0         0.0        0.0       …   0.0        0.0       0.0
 1.27008     1.0         0.0        0.0           0.0        0.0       0.0
 1.15972     0.272727    1.0        0.0           0.0        0.0       0.0
 0.207019   -0.0726137   1.71374    1.0           0.0        0.0       0.0
 0.784952   -0.151583    2.24811    0.731533      0.0        0.0       0.0
 0.279962    0.14247     0.697592   0.400667  …   0.0        0.0       0.0
 0.712069    1.99265    -3.05642   -0.993996      0.0        0.0       0.0
 0.0869193   0.617841    0.88225    0.739588      1.0        0.0       0.0
 0.292839    1.16701    -1.65242   -0.345392     -0.339624   1.0       0.0
 0.64728     0.150725    1.06299    0.507226     -0.667365  -0.329313  1.0

10×10 Matrix{Float64}:
  0.708457     0.167625  0.224108  …   0.561845    0.247075    0.0182439
 -1.11022e-16  0.417013  0.518452     -0.291677    0.118201    0.441063
  0.0          0.0       0.385658      0.0990851   0.608821    0.602141
  0.0          0.0       0.0           0.206955   -0.637261   -0.731066
  0.0          0.0       0.0          -0.474726   -0.264278   -0.527257
  0.0          0.0       0.0       …   0.471052   -0.232717   -0.0461401
  0.0          0.0       0.0           1.9368     -0.0593531  -0.1737
  0.0          0.0       0.0           1.57539    -0.24736     0.577885
  0.0          0.0       0.0           0.0         1.06798    -0.199146
  0.0          0.0       0.0           0.0         0.0         0.320097

# Factorización LU por bloques con LU paralelo

## Doolittle

In [87]:
using Base.Threads
using LinearAlgebra

function is_singular(A)
    return det(A) == 0
end

function pseudo_parallel_LU_block_LU_Doolittle(A)
    n = size(A, 1)
    if n == 1
        return A, A, A
    end

    m = n ÷ 2
    A11 = A[1:m, 1:m]
    A12 = A[1:m, m+1:end]
    A21 = A[m+1:end, 1:m]
    A22 = A[m+1:end, m+1:end]

    if is_singular(A11) || is_singular(A22)
        error("La matriz es singular")
    end

    L11, U11 = parallel_LU_Doolittle(A11)
    U12 = L11 \ A12
    L21 = A21 / U11
    L22, U22 = parallel_LU_Doolittle(A22 - L21*U12)

    L = [L11 zeros(m, n-m); L21 L22]
    U = [U11 U12; zeros(n-m, m) U22]

    return L, U
end

pseudo_parallel_LU_block_LU_Doolittle (generic function with 1 method)

### Pruebas

In [88]:
A = rand(10,10)
L, U = pseudo_parallel_LU_block_LU_Doolittle(A)

display(L)
display(U)

10×10 Matrix{Float64}:
 1.0        0.0       0.0      …   0.0       0.0       0.0       0.0
 1.08371    1.0       0.0          0.0       0.0       0.0       0.0
 0.105315   0.0       1.0          0.0       0.0       0.0       0.0
 0.81004   17.1421   34.4368       0.0       0.0       0.0       0.0
 0.072129   0.0       0.0          0.0       0.0       0.0       0.0
 1.19638    4.48589   5.96309  …   0.0       0.0       0.0       0.0
 0.167628   1.56894   6.42201      1.0       0.0       0.0       0.0
 0.725697   6.2327   12.8702       0.0       1.0       0.0       0.0
 1.20819    8.70299  13.5023      -0.306635  0.529041  1.0       0.0
 0.664066   6.54464  13.2158       0.0       0.0       0.196421  1.0

10×10 Matrix{Float64}:
 0.697912     0.0452234   0.828605  …    0.145046     0.569717    0.25816
 1.11022e-16  0.0382121  -0.467496       0.678331    -0.535251    0.166349
 0.0          0.0308118   0.24168        0.0475272    0.717635    0.296168
 0.0          0.0         0.0          -13.0627     -15.8726    -13.0373
 0.0          0.530585   -0.011076      -2.82527     -3.30824    -2.18387
 0.0          0.0         0.0       …   -0.509844     0.989458    0.122318
 0.0          0.0         0.0            0.0396188   -2.26301    -0.596636
 0.0          0.0         0.0           -0.534215    -0.463856   -0.277762
 0.0          0.0         0.0            0.0          1.43183     0.299406
 0.0          0.0         0.0            0.25259      0.0         0.434033